<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md)

Lab 3.3 : Key/Value Pair RDDs
=============================
### Overview
Learn to work with 

### Depends On 
[3.2 multi RDD operations](2-rdd-multi.md)

### Run time
20-30 mins


----------------------------
Meetup Recommendation Take 2
----------------------------
In the previous lab we created datasets using `parallelize` keyword.  In this lab we are going to load key/value RDDs from files.


----------------------------
Analyzing Meetup Data
----------------------------

## Step 1 : Start Spark Shell
Change working directory to `spark-labs`.  This way, we can access data using relative paths (makes life simpler)

```bash
    $   cd ~/spark-labs
    $   ~/spark/bin/spark-shell
```

## Step 2: Load data
The file [data/meetup/meetup.csv](/data/meetup/meetup.csv)  has following contents

```
    u1,m1
    u2,m2
    u1,m2
    u3,m1
    u2,m3
    u2,m4
    u3,m5
    u1,m3
    u2,m5
    u4,m6
    u5,m1
```

In [5]:
meetups = sc.textFile("/data/meetup/meetup.csv")

**=> Print the content of variable `meetups`**  
Hint : `collect`  

`meetups` is an RDD of strings.



In [6]:
# TODO: print the contents of meetups
meetups.collect()


['u1,m1',
 'u2,m2',
 'u1,m2',
 'u3,m1',
 'u2,m3',
 'u2,m4',
 'u3,m5',
 'u1,m3',
 'u2,m5',
 'u4,m6',
 'u5,m1',
 'u6,m2',
 'u7,m3',
 'u8,m1']

## Step 3 : Creating a key-value pair RDD 
Use `map` to create pair RDD

In [7]:
userMeetupsKV = meetups.map(lambda line : (line.split(",")[0], line.split(",")[1]))

**=> See the 'RDD lineage' of `userMeetupsKV`**  
Hint : `userMeetupsKV.toDebugString`

In [8]:
# TODO find the lineage of RDD.

userMeetupsKV.toDebugString()

b'(2) PythonRDD[5] at RDD at PythonRDD.scala:48 []\n |  /data/meetup/meetup.csv MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  /data/meetup/meetup.csv HadoopRDD[3] at textFile at NativeMethodAccessorImpl.java:-2 []'

 read from bottom up
```
b'(2) PythonRDD[5] at RDD at PythonRDD.scala:48 []\n 
|  /data/meetup/meetup.csv MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2 []\n 
|  /data/meetup/meetup.csv HadoopRDD[3] at textFile at NativeMethodAccessorImpl.java:-2 []'
```

**=> Print the contents of `userMeetupsKV`**

**=> What is resulting RDD type from this?**

In [ ]:
# TODO: Print contents of userMeetupsKV and find the RDD type



In [10]:
x = meetups.map(lambda line: line.split(","))

## Step 4 : RDD Operations

**=> Find all meetups per user**  
Hint : `groupByKey`


In [ ]:
# TODO: find meetups by user



**=> Print the lineage of grouped RDD**  
Hint : `toDebugString`

In [ ]:
userMeetupsKV.groupByKey().toDebugString()

**=> See DAG for groupby execution**  
Execute the following in Shell

In [11]:
userMeetupsKV.groupByKey().collect()

[('u8', <pyspark.resultiterable.ResultIterable at 0x1095162b0>),
 ('u5', <pyspark.resultiterable.ResultIterable at 0x1095162e8>),
 ('u6', <pyspark.resultiterable.ResultIterable at 0x109516470>),
 ('u7', <pyspark.resultiterable.ResultIterable at 0x109516390>),
 ('u2', <pyspark.resultiterable.ResultIterable at 0x1095164e0>),
 ('u1', <pyspark.resultiterable.ResultIterable at 0x109516588>),
 ('u4', <pyspark.resultiterable.ResultIterable at 0x109516828>),
 ('u3', <pyspark.resultiterable.ResultIterable at 0x1095166d8>)]

And inspect the **Spark Shell UI (port 4040)** and look at the **DAG** for this execution.  You might see something like this.  
Notice how `groupByKey` is done in another stage.

<img src="../assets/images/3.3a.png" style="border: 5px solid grey; max-width:100%;"/>

**=> Find meetups for user 'u1'**  
Hint : `filter`  
`rdd.filter{case (k,v) => k == "u1"}`

**=> Find number of meetups per user**  
Hint : `countByKey`

**=> Find all unique meetups in the dataset**  
Hint : `values` and `distinct`

**=> Sort the output of above**  
Hint : `sorted`


----------------------------
More RDD Operations
----------------------------
## Step 5 : Create an RDD with meetup as key
For some operations we need to swap key and value pair.
Here is how...

In [ ]:
#from original dataset
meetups = sc.textFile("data/meetup/meetup.csv")
meetupUsersKV = meetups.map(lambda line : (line.split(",")[1], line.split(",")[0]))


In [14]:

#swapping an already existing KV RDD
meetupUsersKV = userMeetupsKV.map(lambda kv:(kv(1), kv(0)))

**=> Find all users for each meetup**

**=> Find number of users for each meetup**

**=> Sort the meetup by most popular to least**  
Hint : We need to sort the results.  One option is converting the Map to a List and sorting it. Try this  
`meetupUserCount.toSeq.sortWith((a,b) => a._2 > b._2)`



----------------
Join Operations
----------------
## Step 6 : Load Users Dataset
The file [data/meetup/users.csv](/data/meetup/users.csv) contains users data in the following format

In [ ]:
```
   userid, gender, languages(note - separated by ;)

    u1,M,Java;Ruby
    u2,F,Scala;Python;Erlang
    u3,M,Java;Scala
    u4,F,C++;PHP;Go
    u5,M,Scala;Python
    u6,M,Go;Shell;C++
    u7,F,PHP;Ruby
    u8,F,Python;Erlang
    u9,M,Java;C++
    u10,M,Java
```

**=> Load the user data set**

In [ ]:
    // ===== Scala =====
    val users = sc.textFile("data/meetup/users.csv")
    // create user RDD : user -> gender
    val usersKV = users.map(line => {
                        val tokens = line.split(",")
                        val userName = tokens(0) // first
                        val gender = tokens(1) // second
                        val langs = tokens(2).split(";") // third, create lang array
    
                        // TODO : just return user and gender
                        (???, ???)
                        })
    usersKV.collect

## Step 7 : Join User data and Meetup data

In [ ]:
    // ===== Scala =====
    val joinedUsersMeetups = usersKV.join(userMeetupsKV)
    joinedUsersMeetups.collect
    //  Array[(String, (String, String))] = Array((u5,(M,m1)), (u3,(M,m1)), 
    // ...

**=> Inspect the RDD type for `joinedUsersMeetups`**


## Step 8 : Meetups and Gender
Using `joinedUsersMeetups` RDD, create a new RDD as gender as key and meetup as value

In [ ]:
    // ===== Scala =====
    // gender -> meetup
    val genderMeetups = joinedUsersMeetups.map{
        case (user, (gender, meetup)) => (???, ???)  // TODO : return the correct values
        }
    genderMeetups.collect

**=> Calculate how many Male / Females attend meetups (overall distribution)** 

## Bonus Lab 
Create an RDD with meetup as key and gender as value.  
What kind of interesting facts you can find from this RDD?